# Machine Learning Assingment - 168899



In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Module Imports

In [64]:
#required imports to run modules
%matplotlib inline
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.constraints import max_norm
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from keras.optimizers import Adam
#!pip install -q git+https://github.com/tensorflow/docs
#import tensorflow_docs as tfdocs
#import tensorflow_docs.modeling
#import tensorflow_docs.plots
from sklearn.impute import KNNImputer
np.set_printoptions(precision=3,suppress=True)
import matplotlib.pyplot as plt
from sklearn import model_selection

Reading data from csv files


In [4]:
#loading data from csv files using pandas
training_data         = pd.read_csv("/content/drive/MyDrive/ML_csv_files/training.csv")
test_data             = pd.read_csv("/content/drive/MyDrive/ML_csv_files/testing.csv")
additional_data       = pd.read_csv("/content/drive/MyDrive/ML_csv_files/additional_training.csv")
annotation_confidence = pd.read_csv("/content/drive/MyDrive/ML_csv_files/annotation_confidence.csv")
test_proportion       = pd.read_csv("/content/drive/MyDrive/ML_csv_files/test_proportions.csv")

In [5]:
#adding together trianing and additional data
frames = [additional_data,training_data]
train=pd.concat(frames)

In [ ]:
#train=train.replace(np.NaN, train.mean())

Seperating features of data

In [6]:
#Seperation of CNN and GIST features in training data
training_CNN      = train.iloc[:,1:4097]
training_GIST     = train.iloc[:,4097:4608]
training_labels   = train.iloc[:,4609]
#Seperataion of CNN and GIST features in test data
testing_CNN       = test_data.iloc[:,1:4097]
testing_GIST      = test_data.iloc[:,4097:4608]

Putting data into arrays

In [7]:
training_CNN = np.asarray(training_CNN)
training_GIST=np.asarray(training_GIST)
training_labels = np.asarray(training_labels)
testing_CNN  = np.asarray(testing_CNN)
testing_GIST = np.asarray(testing_GIST)
confidence=np.asarray(annotation_confidence)

Pre-processing

In [8]:
#KNN
#Using kNN to impute missing values of CNN and GIST features
imputer = KNNImputer(n_neighbors=2)
training_CNN = imputer.fit_transform(training_CNN)
training_GIST = imputer.fit_transform(training_GIST)

In [112]:
normalize = preprocessing.Normalization()
normalize.adapt(training_CNN)
normalize.adapt(testing_CNN)

In [114]:
normalize = preprocessing.Normalization()
normalize.adapt(training_GIST)

Creating the model and compiling it

In [115]:
class MCDropout(tf.keras.layers.Layer):
    def __init__(self, rate):
        super(MCDropout, self).__init__()
        self.rate = rate

    def call(self, inputs):
        return tf.nn.dropout(inputs, rate=self.rate)

Basic model for CNN and GIST comaparison

In [116]:
featurecompmodel=Sequential([
                             normalize,
                             layers.Dense(16,activation='relu'),
                             layers.Dense(1,activation='sigmoid'),
                              ])

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
featurecompmodel.compile(optimizer=opt,
                          loss=keras.losses.BinaryCrossentropy(from_logits=True),
                          metrics=['binary_accuracy'],
                          )

In [106]:
#CNN
historyCNN = featurecompmodel.fit(training_CNN,
                               training_labels,
                               epochs=10,
                               verbose=2,
                               batch_size=256,                 
                               validation_split=0.05,
                               )
#val accuracy of 0.6626 after 10 epochs

Epoch 1/10
25/25 - 0s - loss: 0.0714 - binary_accuracy: 0.9732 - val_loss: 2.9029 - val_binary_accuracy: 0.6778
Epoch 2/10
25/25 - 0s - loss: 0.0702 - binary_accuracy: 0.9745 - val_loss: 2.8638 - val_binary_accuracy: 0.6748
Epoch 3/10
25/25 - 0s - loss: 0.0645 - binary_accuracy: 0.9750 - val_loss: 2.9624 - val_binary_accuracy: 0.6717
Epoch 4/10
25/25 - 0s - loss: 0.0623 - binary_accuracy: 0.9751 - val_loss: 2.9821 - val_binary_accuracy: 0.6748
Epoch 5/10
25/25 - 0s - loss: 0.0621 - binary_accuracy: 0.9768 - val_loss: 3.1224 - val_binary_accuracy: 0.6809
Epoch 6/10
25/25 - 0s - loss: 0.0658 - binary_accuracy: 0.9747 - val_loss: 3.0967 - val_binary_accuracy: 0.6748
Epoch 7/10
25/25 - 0s - loss: 0.0627 - binary_accuracy: 0.9758 - val_loss: 3.0777 - val_binary_accuracy: 0.6809
Epoch 8/10
25/25 - 0s - loss: 0.0576 - binary_accuracy: 0.9772 - val_loss: 3.1605 - val_binary_accuracy: 0.6687
Epoch 9/10
25/25 - 0s - loss: 0.0543 - binary_accuracy: 0.9790 - val_loss: 3.2710 - val_binary_accuracy:

In [117]:
historyCNN = featurecompmodel.fit(training_GIST,
                               training_labels,
                               epochs=10,
                               verbose=2,
                               batch_size=256,                 
                               validation_split=0.05,
                               )
#val accuracy of 0.6748 after 10 epochs

Epoch 1/10
25/25 - 1s - loss: 0.8284 - binary_accuracy: 0.6184 - val_loss: 0.6237 - val_binary_accuracy: 0.6778
Epoch 2/10
25/25 - 0s - loss: 0.6111 - binary_accuracy: 0.6825 - val_loss: 0.6028 - val_binary_accuracy: 0.7082
Epoch 3/10
25/25 - 0s - loss: 0.5766 - binary_accuracy: 0.7079 - val_loss: 0.5982 - val_binary_accuracy: 0.7021
Epoch 4/10
25/25 - 0s - loss: 0.5631 - binary_accuracy: 0.7132 - val_loss: 0.6124 - val_binary_accuracy: 0.6930
Epoch 5/10
25/25 - 0s - loss: 0.5375 - binary_accuracy: 0.7258 - val_loss: 0.6080 - val_binary_accuracy: 0.6960
Epoch 6/10
25/25 - 0s - loss: 0.5216 - binary_accuracy: 0.7488 - val_loss: 0.6324 - val_binary_accuracy: 0.6809
Epoch 7/10
25/25 - 0s - loss: 0.5120 - binary_accuracy: 0.7542 - val_loss: 0.6853 - val_binary_accuracy: 0.6413
Epoch 8/10
25/25 - 0s - loss: 0.5007 - binary_accuracy: 0.7574 - val_loss: 0.6998 - val_binary_accuracy: 0.6383
Epoch 9/10
25/25 - 0s - loss: 0.4895 - binary_accuracy: 0.7706 - val_loss: 0.6739 - val_binary_accuracy:

Basic Model for SELU comparison

In [95]:
featureactivationmodel=Sequential([
                             normalize,
                             layers.Dense(64,activation='selu',kernel_initializer='lecun_normal'),
                             layers.Dense(1,activation='sigmoid'),
                              ])

opt = tf.keras.optimizers.Adam(learning_rate=0.02)
featureactivationmodel.compile(optimizer=opt,
                          loss=keras.losses.BinaryCrossentropy(from_logits=True),
                          metrics=['binary_accuracy'],
                          )

In [123]:
historyCNN = featureactivationmodel.fit(training_CNN,
                               training_labels,
                               epochs=10,
                               verbose=2,
                               batch_size=256,                 
                               validation_split=0.05,
                               )

Epoch 1/10
25/25 - 1s - loss: 0.1260 - binary_accuracy: 0.9315 - val_loss: 1.8333 - val_binary_accuracy: 0.6778
Epoch 2/10
25/25 - 0s - loss: 0.1236 - binary_accuracy: 0.9309 - val_loss: 2.0304 - val_binary_accuracy: 0.6839
Epoch 3/10
25/25 - 0s - loss: 0.1163 - binary_accuracy: 0.9388 - val_loss: 1.9488 - val_binary_accuracy: 0.6960
Epoch 4/10
25/25 - 0s - loss: 0.1085 - binary_accuracy: 0.9412 - val_loss: 2.2311 - val_binary_accuracy: 0.6900
Epoch 5/10
25/25 - 0s - loss: 0.0959 - binary_accuracy: 0.9445 - val_loss: 2.4169 - val_binary_accuracy: 0.6717
Epoch 6/10
25/25 - 0s - loss: 0.0978 - binary_accuracy: 0.9448 - val_loss: 2.4488 - val_binary_accuracy: 0.6413
Epoch 7/10
25/25 - 0s - loss: 0.1962 - binary_accuracy: 0.9254 - val_loss: 2.9137 - val_binary_accuracy: 0.6505
Epoch 8/10
25/25 - 0s - loss: 0.3207 - binary_accuracy: 0.8945 - val_loss: 1.6685 - val_binary_accuracy: 0.6869
Epoch 9/10
25/25 - 0s - loss: 0.2373 - binary_accuracy: 0.9009 - val_loss: 1.9359 - val_binary_accuracy:

SELU small model

In [19]:
#small selu Model
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=0)
classifier_smallselu_model = Sequential([
                    normalize,

                    layers.Dense(4069,activation=None,kernel_regularizer=regularizers.l2(0.001), input_shape=(4096,)),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(16,activation='selu',kernel_regularizer=regularizers.l2(0.001),kernel_initializer='lecun_normal'),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(16,activation='selu',kernel_regularizer=regularizers.l2(0.001),kernel_initializer='lecun_normal'),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(16,activation='selu',kernel_regularizer=regularizers.l2(0.001),kernel_initializer='lecun_normal'),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(1,activation='sigmoid'),
                    ])
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
classifier_smallselu_model.compile(optimizer=opt,
                          loss=keras.losses.BinaryCrossentropy(from_logits=True),
                          metrics=['binary_accuracy'],
                          
                          )


SELU Large model

In [28]:
#Large selu model
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=0)
classifier_largeselu_model = Sequential([
                    normalize,

                    layers.Dense(4069,activation=None,kernel_regularizer=regularizers.l2(0.001), input_shape=(4096,)),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(128,activation='selu',kernel_regularizer=regularizers.l2(0.001),kernel_initializer='lecun_normal'),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(128,activation='selu',kernel_regularizer=regularizers.l2(0.001),kernel_initializer='lecun_normal'),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(128,activation='selu',kernel_regularizer=regularizers.l2(0.001),kernel_initializer='lecun_normal'),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(1,activation='sigmoid'),
                    ])
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
classifier_largeselu_model.compile(optimizer=opt,
                          loss=keras.losses.BinaryCrossentropy(from_logits=True),
                          metrics=['binary_accuracy'],
                          
                          )


SELU Huge model



In [97]:
#Large selu model
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
classifier_hugeselu_model = Sequential([
                    normalize,

                    layers.Dense(4069,activation=None,kernel_regularizer=regularizers.l2(0.001), input_shape=(4096,)),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(1024,activation='selu',kernel_regularizer=regularizers.l2(0.001),kernel_initializer='lecun_normal'),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(512,activation='selu',kernel_regularizer=regularizers.l2(0.001),kernel_initializer='lecun_normal'),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(512,activation='selu',kernel_regularizer=regularizers.l2(0.001),kernel_initializer='lecun_normal'),
                    MCDropout(rate=0.5),
                    layers.BatchNormalization(),
                    layers.Dense(1,activation='sigmoid'),
                    ])
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
classifier_hugeselu_model.compile(optimizer=opt,
                          loss=keras.losses.BinaryCrossentropy(from_logits=True),
                          metrics=['binary_accuracy'],
                          
                          )

small selu fit

In [33]:
history1 = classifier_smallselu_model.fit(training_CNN,
                               training_labels,
                               epochs=30,
                               verbose=2,
                               batch_size=256,                 
                               validation_split=0.05,
                               callbacks=callback
                               )

Epoch 1/30
25/25 - 12s - loss: 0.8629 - binary_accuracy: 0.7282 - val_loss: 0.8629 - val_binary_accuracy: 0.7143
Epoch 2/30
25/25 - 12s - loss: 0.8833 - binary_accuracy: 0.7250 - val_loss: 0.8368 - val_binary_accuracy: 0.7568
Epoch 3/30
25/25 - 13s - loss: 0.8801 - binary_accuracy: 0.7273 - val_loss: 0.8873 - val_binary_accuracy: 0.6960


Large selu fit

In [34]:
history3 = classifier_largeselu_model.fit(training_CNN,
                               training_labels,
                               epochs=30,
                               verbose=2,
                               batch_size=256,                 
                               validation_split=0.05,
                               callbacks=callback
                               )

Epoch 1/30
25/25 - 14s - loss: 10.5836 - binary_accuracy: 0.6938 - val_loss: 5.4731 - val_binary_accuracy: 0.7295
Epoch 2/30
25/25 - 12s - loss: 2.5990 - binary_accuracy: 0.7444 - val_loss: 1.4976 - val_binary_accuracy: 0.7356
Epoch 3/30
25/25 - 12s - loss: 1.2002 - binary_accuracy: 0.7420 - val_loss: 1.0448 - val_binary_accuracy: 0.7386
Epoch 4/30
25/25 - 12s - loss: 0.9577 - binary_accuracy: 0.7404 - val_loss: 0.9154 - val_binary_accuracy: 0.7264
Epoch 5/30
25/25 - 12s - loss: 0.8832 - binary_accuracy: 0.7346 - val_loss: 0.8537 - val_binary_accuracy: 0.7356
Epoch 6/30
25/25 - 12s - loss: 0.8686 - binary_accuracy: 0.7414 - val_loss: 0.8318 - val_binary_accuracy: 0.7295
Epoch 7/30
25/25 - 12s - loss: 0.8801 - binary_accuracy: 0.7311 - val_loss: 0.9058 - val_binary_accuracy: 0.7234


SELU HUGE TEST


In [98]:

history4 = classifier_hugeselu_model.fit(training_CNN,
                               training_labels,
                               epochs=30,
                               verbose=2,
                               batch_size=256,                 
                               validation_split=0.05,
                               callbacks=callback
                               )

Epoch 1/30
25/25 - 19s - loss: 17.9840 - binary_accuracy: 0.6891 - val_loss: 15.7409 - val_binary_accuracy: 0.6261
Epoch 2/30
25/25 - 17s - loss: 5.7332 - binary_accuracy: 0.7194 - val_loss: 2.5500 - val_binary_accuracy: 0.6991
Epoch 3/30
25/25 - 17s - loss: 1.7803 - binary_accuracy: 0.7314 - val_loss: 1.2546 - val_binary_accuracy: 0.6748
Epoch 4/30
25/25 - 17s - loss: 1.1138 - binary_accuracy: 0.7407 - val_loss: 1.1193 - val_binary_accuracy: 0.7234
Epoch 5/30
25/25 - 17s - loss: 1.0287 - binary_accuracy: 0.7343 - val_loss: 1.0448 - val_binary_accuracy: 0.7082
Epoch 6/30
25/25 - 17s - loss: 1.0254 - binary_accuracy: 0.7340 - val_loss: 1.0376 - val_binary_accuracy: 0.7082
Epoch 7/30
25/25 - 17s - loss: 1.0771 - binary_accuracy: 0.7343 - val_loss: 1.0550 - val_binary_accuracy: 0.7325
Epoch 8/30
25/25 - 17s - loss: 1.0746 - binary_accuracy: 0.7346 - val_loss: 1.0721 - val_binary_accuracy: 0.7143


Prediction Model

In [119]:
prediction_model = Sequential([classifier_largeselu_model])
predictions = prediction_model.predict(testing_CNN,verbose=1)
predictions = [round(x[0]) for x in predictions]

141/141 [==============================] - 5s 34ms/step


In [120]:
test_prob=predictions
test_prediction_df=pd.DataFrame()
test_prediction_df['ID']=np.arange(1,len(test_prob)+1)
test_prediction_df['prediction']=np.array(test_prob)
test_prediction_df.to_csv("/content/drive/MyDrive/Colab_Notebooks/Predictions/test12.csv",index=None)

Unused Methods

In [ ]:
#splitting training data for model testing         DO NOT USE!

#modeltrain_CNN,    modeltest_CNN    = model_selection.train_test_split(training_CNN, train_size = 0.9)
#modeltrain_GIST,   modeltest_GIST   = model_selection.train_test_split(training_GIST, train_size = 0.9)
#modeltrain_labels, modeltest_labels = model_selection.train_test_split(training_labels, train_size = 0.9)

In [ ]:
#Mean
#modeltrain=np.append()
#modeltest_CNN=imputer.fit_transform(modeltest_CNN)
#modeltest_GIST=imputer.fit_transform(modeltest_GIST)

In [100]:
#scaling = StandardScaler()
#training_CNN = scaling.fit_transform(training_CNN)
#training_GIST = scaling.fit_transform(training_GIST)
#test_CNN = scaling.fit_transform(testing_CNN)